# Utils

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
def extract_file_name(file_path):
  """
  Extracts the file name from a file path.

  Args:
    file_path (str): The file path.

  Returns:
    str: The file name without the extension.
  """
  # Split the file path into a list of strings
  parts = file_path.split("/")
  # Get the last element in the list (the file name)
  file_name = parts[-1]
  # Split the file name into a list of strings
  parts = file_name.split(".")
  # Get the first element in the list (the file name without the extension)
  file_name = parts[0]

  return file_name

def read_images_and_names(dir_path, func=None):
    """
    Read all images and their corresponding names in a directory and return them as a list of tuples.
    Each tuple in the list contains a NumPy array representing the image and a string representing the name of the image.

    Parameters:
        dir_path (str): The path to the directory containing the images and their names.

    Returns:
        images_and_names: A list of tuples, where each tuple contains a NumPy array representing the image and a string representing the name of the image.
    """
    images_and_names = []
    for filename in os.listdir(dir_path):
        # Check if file is an image
        if filename.endswith(".png") or filename.endswith(".jpg") or filename.endswith(".PNG") or filename.endswith(
                ".JPG") or filename.endswith(".jpeg"):
            # Read image and store as NumPy array
            file_path = os.path.join(dir_path, filename)
            image = cv2.imread(file_path)
            if func is not None:
              image = func(image)
            image_name = extract_file_name(file_path)
            images_and_names.append((image, image_name))
    return images_and_names

In [3]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
import os
import pickle
import json
import pandas as pd

# Read Images

In [4]:
patches_path = "/content/drive/MyDrive/Thesis: Cannabis maturity assessment project/experiments/filter relevant areas using SAM"
images_names = read_images_and_names(patches_path)
images, names = zip(*images_names)

# SAM

In [5]:
using_colab = True

In [6]:
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install opencv-python matplotlib
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

    !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

PyTorch version: 2.0.1+cu118
Torchvision version: 0.15.2+cu118
CUDA is available: True
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-dd717jy5
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-dd717jy5
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36588 sha256=f9584d1f27dc1032f9b4538d273528949f65d0632292890dfe21e57573e761f9
  Stored in directory: /tmp/pip-ephem-wheel-cache-23q2npjk/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment-anything
--2023-07-29 13:10:21--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.224.249.94, 13.224.249.63, 

In [7]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
import os

In [8]:
def show_anns(anns):
    # If there are no annotations, exit the function early
    if len(anns) == 0:
        return

    # Sort the annotations by area in descending order
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)

    # Get the current axis (plot) on which to draw
    ax = plt.gca()
    # Disable autoscaling of the axis
    ax.set_autoscale_on(False)

    # Create a transparent image of the same size as the first (largest) annotation
    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0

    # Loop over each annotation
    for ann in sorted_anns:
        # Get the mask for this annotation
        m = ann['segmentation']

        # Generate a random color for this mask (RGB + alpha, where alpha is 0.35)
        color_mask = np.concatenate([np.random.random(3), [0.35]])

        # Apply the color to the mask on the image
        img[m] = color_mask

    # Display the image with the colored masks
    ax.imshow(img)

In [9]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

In [10]:
def segments_with_SAM(images_names):
    segmentation_dict = {}

    for image, name in images_names:
        mask = mask_generator.generate(image)
        num_segments = len(mask)

        # Initialize an array of False with the same shape as the segment masks
        instance_bitmap = np.zeros_like(mask[0]['segmentation'], dtype=bool)

        # create a single instance bitmap
        for seg in mask:
          instance_bitmap = np.logical_or(instance_bitmap, seg['segmentation'])


        segmentation_dict[name] = {
            'mask': mask,
            'num_segments': num_segments,
            "instance_bitmap": instance_bitmap,
        }

    return segmentation_dict

Run the SAM model on the image

In [11]:
mask = mask_generator.generate(images_names[0][0])

OutOfMemoryError: ignored

In [ ]:
SAM_pred = segments_with_SAM(images_names)